# 문제점 : 분류 문제에서 잔차를 구한다는 것은 어떠한 의미인가? 

예시로 이진 분류에서 y값이 -1 / +1 둘 중 하나를 띈다고 가정할 때, 
y 값의 list가 [1,1,-1-1] 이나, $f_1(x)$ 가 [1, -1, 1, -1] 로 예측했다고 하자. 
그럼 잔차 (y - f(x)) = [0 , 2, -2, 0] 이 된다. 

이때 -2, 0, 2 는 모두 기존 분류에서 없던 값들이다. 그렇다면 잔차를 다시 추정하는 $f_2(x)$ 는 $f_1(x)$와 달리 3분류를 진행하는 것인가? 

$f_n(x)$ 함수는 계속해서 Class의 개수가 늘어날 텐데 이렇게 계산하는 것이 맞나?


##### 해결책 :  잔차를 구할 때, f(x)를 DT 회귀모델로 적용하여 1, -1 외의 다른 값들도 적용할 수 있도록 한다. 
알고리즘 구현 간 크게 실수 한 부분 중 하나는 회귀모델을 사용해야 하는데, 결정 모델로 잔차를 구하고자 한 점. 

클래스별로 One-hot incoding 을 진행하며, 클래스별 개수 만큼의 Dataset를 분리한다. 
각 클래스 별로 Decision Tree Regression을 통해서 예측값을 구한다. 

##### 고민점
위의 방식대로 풀이하면 비교적 정확한 예측을 할 수 있음. 하지만 f(x)를 DT Regression을 적용하여 약한 모델을 적용했다기엔 어폐가 있음. 

즉, 현재 GBM 모델이 정확히 분류하는 것은 첫 번째 f(x) 과정에서 정확한 답을 찾아서 인지, M번의 Gradient 를 더하는 과정에서 정확해진 것인지 구분이 가지 않음. 
 Target Value로 유리수를 받아들일 수 있으면서, 비교적 약한 모델이 있다면 적용할 수 있을 듯함. 추후 시도해볼 것. 

# Ensemble Learning - GBM (이번 구현에서는 미사용) 

### 수도 알고리즘은 강의 자료 참고 

1. Initialize $f_0(x)$ = $argmin_\gamma \sum_{i=1}^N L(y_i, \gamma)$ 
2. For m = 1 to M 
- for i = 1, ... N compute 
> $g_{im} = [\frac{\partial L(y_i, f(x_i))}{\partial f(x_i)}]_{f(x_i) = f_{m-1}(x_i)}$ 
- Fit a regression tree to the targets $g_{im}$ giving terminal regions $R_{jm}, j=1, ... , J_m$ 
- For j=1, ..., $J_m$ compute 
> $\gamma_jm$ = $argmin_\gamma \sum_{x_i \in R_{jm}} L(y_i, f_{m-1}(x_i) + \gamma)$
- update $f_m(x) = f_{m-1}(x) + \sum_{j=1}^{J_m} \gamma_{jm} I(x \in R_{jm})$ 
3. Output $\hat f(x) = f_M(x)$ 

**구현해야 하는 것**
- $f_0(x)$ = $argmin_\gamma \sum_{i=1}^N L(y_i, \gamma)$  : 임의의 Stump tree 진행. 
- $\gamma_jm$ = $argmin_\gamma \sum_{x_i \in R_{jm}} L(y_i, f_{m-1}(x_i) + \gamma)$ : y- $f_1(x)$ 로 Dataset의 값을 변경시키고, 이를 예측하는 Stump tree 적용
* 원래는 함수를 더해가는 게 맞으나, 함수를 더하는 과정을 어떻게 구현할지 감이 안옴. 이에 데이터를 변경시키는 쪽으로 구현하겠음. 

- f(x) : 약한 학습기. Stump tree 적용 
- loss 함수 : OLS 적용
- Aggregation 함수 : Majority 함수 채택 

**필요한 것** 
- Data : Stump Tree를 적용할 것으로 y값이 1, -1로 구분될 것 
- M : 트리 개수
- Aggregating 방식 


### GBM 알고리즘 변경안

1. 클래스 레이블을 기준으로 one-hot encoding을 진행한다. 
2. 각 클래스 별 one-hot encoding 렬을 y 값으로 하여 Decision tree regression을 진행한다. 
3. 잔차를 계산한 후, -잔차를 y값에 부여한다.
4. 갱신된 y값에 따라 새롭게 Decision tree regression을 진행하여 y 값을 예측한다. 

위의 과정을 M번 반복하면서 정확도를 향상시킨다. 

**구현해야 하는 것**
- One - hot encoding 
- Decision tree regression : 모듈 사용 

**필요한 것**
- M : 반복 학습 횟수 
- X : Input data
- y : output data 

**함수의 형태** 
- Softmax는 개별 구현 
- 그 외 One-hot encoding 등 다른 기능은 Class 내에서 구현할 것. 


In [42]:
import numpy as np
from sklearn import tree

from sklearn.datasets import load_iris
X = load_iris()['data']

# y의 값을 +1, -1 둘 중 하나로 변경 
y = load_iris()["target"]

In [47]:
class GBM() : 
    def __init__(self, X, y, M) : 
        self.X = X 
        self.n = np.shape(X)[0]
        self.d = np.shape(X)[1]
        
        self.y = y 
        self.num_label = len(np.unique(y))
        self.one_hot_y = self.one_hot(y)
        
        self.M = M 
        
        self.model_lst = self.modeling()
    
    
    def one_hot(self, y) : 
        dataset = [] 
        
        for label in range(self.num_label) : 
            data = [1 if y[i] == label else 0 for i in range(len(y))]
            dataset.append(data)
        return dataset
    
    def modeling(self) : 
        model_lst = []
        for num in range(self.M) : 
            model_set = [] 
            pred_y_lst = [] 

            # 각 label 별로 DT Regression 진행 
            for label in range(self.num_label) : 
                model = tree.DecisionTreeRegressor()
                model.fit(self.X, self.one_hot_y[label]) 
                pred_y = model.predict(X)
                pred_y_lst.append(pred_y)
                model_set.append(model)
            
            # 각 레이블 별 예측 결과를 모아서 Softmax 진행 (불필요한 과정으로 판단)
            """
            pred_y_lst = np.array(pred_y_lst).swapaxes(0,1)
            for i in range(np.shape(pred_y_lst)[0]) : 
                pred_y_lst[i] = softmax(pred_y_lst[i])
            
            pred_y_lst = pred_y_lst.swapaxes(0,1)
            """
            
            grad = pred_y_lst - np.array(self.one_hot_y) 
            self.one_hot_y = grad 
            
            model_lst.append(model_set)
        return model_lst
    
    def test(self) : 
        solution_set = [] 
        for i in range(self.num_label) :
            lst = [] 
            for num in range(self.M) : 
                lst.append(self.model_lst[num][i].predict(self.X))
            
            solution_set.append(lst)
        return solution_set
        
    def classify(self, new_x) : 
        new_x = np.reshape(new_x, (1,-1))
        label_predict = [] 
        for i in range(self.num_label):
            predict = [self.model_lst[j][i].predict(new_x) for j in range(self.M)]
            label_predict.append(np.sum(predict)) 
        solution = np.argmax(label_predict) 
        return solution
        
        
            

In [48]:
a = GBM(X,y, 20)
b = a.test()
a.model_lst

c = a.classify([100,2,32,41])

print(c)

2
